## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [2]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('data/2ch_corpus.txt').read()
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('data/lenta.txt').read()

По длине оно сопоставимы.

In [3]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11638405
Длина 2 - 11536552


Напишем простую функцию для нормализации. 

In [4]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [5]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [6]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


И по уникальным токенам

In [7]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [8]:
from collections import Counter

In [9]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


In [10]:
vocab_dvach.most_common(10)

[('и', 55892),
 ('в', 48853),
 ('не', 46602),
 ('на', 29660),
 ('что', 26668),
 ('я', 21734),
 ('а', 21310),
 ('с', 21080),
 ('это', 17727),
 ('ты', 15469)]

In [11]:
vocab_news.most_common(10)

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [12]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796),
 ('как', 0.007882444897390503),
 ('у', 0.006848522895562581),
 ('но', 0.005786090037284669),
 ('так', 0.005383172462170666),
 ('по', 0.005060945990217011),
 ('то', 0.005049649235774562),
 ('все', 0.0046537248896011225),
 ('за', 0.004583792600195488),
 ('же', 0.004228751746289958),
 ('если', 0.004209385881531474)]

In [13]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

In [14]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [15]:
prob.most_common()

[('dvach', 0.034328147047162874), ('news', 0.014186582582287426)]

In [16]:
phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [17]:
prob.most_common()

[('news', 0.05619313197267346), ('dvach', 0.04750823183737408)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова независимыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [18]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [19]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [20]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))


In [21]:
len(unigrams_dvach)

189517

In [22]:
bigrams_news.most_common(10)

[('<start> в', 7972),
 ('<start> по', 6211),
 ('<start> как', 3738),
 ('риа новости', 3504),
 ('по словам', 1971),
 ('об этом', 1795),
 ('<start> однако', 1694),
 ('<start> на', 1643),
 ('что в', 1624),
 ('<start> об', 1619)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [25]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
# phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in bigrams_dvach:
        prob['dvach'] += np.log(bigrams_dvach[ngram]/unigrams_dvach[word1])
    else:
        prob['dvach'] += np.log(0.001)
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'] += np.log(bigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

In [29]:
prob.most_common()

[('news', 3.3368639658312657e-41), ('dvach', 4.888715771763858e-53)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [30]:
matrix_dvach = np.zeros((len(unigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])



In [31]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [32]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [33]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

они потеряли со всеми 
 молодец 
 а остальные 4 имя той толпы следовательно вставлять как только стареть и фаерфаксе свг спрайт 
 и пропустить что это чтоб задыхалась так будь ты хуй на 10 назад 
 от 20 черепов – я посмотрю как худая смотрит 
 с четвёртого энергоблока 
 одним выстрелом расхуярить чтобы было видно что совершил преступление перед прыжком скажу только малая зона в дни месяцы слабо подумать что была мечта казуала 
 с крайностями 
 блин сделали потому что он спит а не дна хуже чем китай 
 год поставил 
 св 
 геноцида ксеносов в


In [34]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

иначе представители таких как сми сказал генерал колонна из юар нельсон автор сюжета было совершено покушение на своих наблюдателей передает риа новости 
 по полному расчету с первым шагом на 15 сентября 
 на средства буквально за рубежом 
 полностью поддержала точку зрения на внешних врагов то будут очень похож на политический советник президента около ста семей 
 владелец предприятия 
 касаясь ситуации в длинном перечне 85 лет и оружия был обнаружен тайник с 1993 по умолчанию не поможет ведь никто не имеет самые решительные действия в пресс-службе всемирного банка 
 в порту сочи началась эпидемия гриппа достигнет рекордного уровня


Попробуйте сделать триграммную модель на основе кода выше.

Подсказки:
    - нужно будет добавить еще один тэг <start>
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похоже на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)